## Scope
In this notebook we give a very brief tutorial which focuses on the mnms python user interface. We'll use a heavily bandlimited version of the `pa6b`(a.k.a. `pa6_f150`) data so that our operations are fast, even though you will probably never use that sim. This way you know how to use mnms in your own analysis code.

## Tutorial

In mnms, we always need to generate a "sqrt-covariance" matrix as a first step; only then can we generate sims:

In [ ]:
from mnms import noise_models as nm, utils, soapack_utils

Typically, we load a `NoiseModel` instance from a config file. The config file stores all of the model parameters, as well as how those parameters map onto filenames (for covariance matrices, sims, etc.). We can start with the `dr6_default_cosmo` config:

In [ ]:
config = utils.config_from_yaml_resource('configs/dr6_default_cosmo.yaml')
print(nm.yaml.dump(config))

Create a `TiledNoiseModel` instance by loading from this config directly:

In [ ]:
tnm = nm.TiledNoiseModel.from_config('dr6_default_cosmo', 'pa6b')

Now we need to make our noise model. Because they are the slowest step to make, and take up a lot of space, by default they are written to disk. The idea is that outside of testing/development, you only need to ever run this command once (per data-release, array-set, other parameters, etc). As you can see, a lot of the time in the below cell is spent loading and preparing the input data.

To keep runtimes fast, we use a low `lmax` of 2000. This cell will take a couple minutes (if it only takes a few seconds, it's probably because someone has already run this notebook and saved models into the same directory. Just change the `lmax` to something unique like 2005 etc. and rerun. Or set `check_on_disk=False`):

In [ ]:
_ = tnm.get_model(0, 2000, keep_model=True)

Now make a `WaveletNoiseModel` instance (again, most of the time here is spent loading input data!):

In [ ]:
wnm = nm.WaveletNoiseModel.from_config('dr6_default_cosmo', 'pa6b')

And make the noise model again (same thing as before -- if this takes only a few seconds, change the `lmax` to something unique):

In [ ]:
_ = wnm.get_model(0, 2000, keep_model=True)

To save you from accidentally regenerating the exact same model again, the arguments `check_in_memory` and `check_on_disk` are `True` by default. If I rerun `get_model`, it will find that the model is already loaded in-memory, and return it in a dictionary. If the model was not yet in memory but was on-disk instead, these cells would take a tiny bit longer as the model would have to be loaded -- but still much shorter than running the model again!

In [ ]:
_ = tnm.get_model(0, 2000)

In [ ]:
_ = wnm.get_model(0, 2000)

That was fast, and that's because the models are in-memory (and on-disk). Let's look at my `covmat_path` from the `mnms` block of my soapack config (if you changed the `lmax` in your models, replace that search string below):

In [ ]:
ls /scratch/gpfs/zatkins/data/ACTCollaboration/mnms/covmats/*lmax2000*

Now we can make sims. Here we see the reason noise model files are only loaded once by default: so that future calls to `get_sim` do not have to spend time loading them from disk! 

By default, sims are ***not*** saved to disk (but they can be, if desired). We need to tell `get_sim` which split, map number, and model `lmax` we want to make. The map number is used in getting a unique random seed (and if the sims is written to disk, will be stored in the filename, according to our config):

In [ ]:
talm_s0_n123_lmax2000 = tnm.get_sim(0, 124, 2000)

In [ ]:
walm_s0_n123_lmax2000 = wnm.get_sim(0, 124, 2000)

As noted in the README, these sims always have shape (num_arrays, num_splits, num_pol, ny, nx), even if some of these axes have dimension 1. Because we generate sims per split, `num_splits=1` always. Let's take a look! 

In [ ]:
shape, wcs = soapack_utils.read_map_geometry(utils.get_data_model('dr6v3'), 'pa6b')
shape, wcs = utils.downgrade_geometry_cc_quad(shape, wcs, 4)

tmap_s0_n123_lmax2000 = utils.alm2map(talm_s0_n123_lmax2000, shape=shape, wcs=wcs)
wmap_s0_n123_lmax2000 = utils.alm2map(walm_s0_n123_lmax2000, shape=shape, wcs=wcs)

utils.eshow(tmap_s0_n123_lmax2000, colorbar=True, downgrade=8, ticks=30)
utils.eshow(wmap_s0_n123_lmax2000, colorbar=True, downgrade=8, ticks=30)

A nice feature is that we can take two models and stitch their sims together in harmonic space. We'll have a transition region from ell=4000 to ell=5000, with a cosine profile. We'll let the `FDWNoiseModel` be the low-ell model by ordering it first when we build a `HarmonicMixture` object. Unlike earlier, we also need to tell the `HarmonicMixture` the `lmaxs` we would like to call on each `NoiseModel` at construction time:

In [ ]:
fnm = nm.FDWNoiseModel.from_config('dr6_default_cosmo', 'pa6b')

hm = nm.HarmonicMixture([fnm, tnm], [5400, 10800], [4000], [5000])
stitched_sim_s0_n123 = hm.get_sim(0, 1)

This was especially fast because we had saved these high-resolution sims to disk ***as alms*** beforehand, and we passed `check_on_disk=True`. Only the "stitching" step was performed on-the-fly. Anyway, we can take a look:

In [ ]:
shape, wcs = soapack_utils.read_map_geometry(utils.get_data_model('dr6v3'), 'pa6b')
shape, wcs = utils.downgrade_geometry_cc_quad(shape, wcs, 2)

stitched_sim_s0_n123 = utils.alm2map(stitched_sim_s0_n123, shape=shape, wcs=wcs)
utils.eshow(stitched_sim_s0_n123, colorbar=True, downgrade=16, ticks=30)